In [ ]:
import numpy as np
from numpy import sqrt, sin, e, ones, linspace

# This is the function we want to integrate
def f(x):
    return sin(sqrt(100*x))**2
# Integration limits
a = 0.0 
b = 1.0 

# Initial number of slices
N = 1 

# Bound of accuracy
acc = 1*e**(-6)

# Implementing the trapezoidal rule
def integ_atrap(N, a, b):
    h= (b-a)/N # edit this
    x = linspace(a,b,N+1) # number of points is number of slices + 1

    weight = ones(N+1) # number of points is number of slices + 1
    weight[0] = 0
    weight[-1] = 0
    weight[::2] = 0 # summing over odd only
    
    I = 0.5*I0 + h*sum(weight*f(x))
    
    return I, (1/3)*(I-I0)


Ivals, Evals, Nvals = list(), list(), list()
I0, E0 = integ_atrap(f,N,0)[0], 1

while abs(E0) > acc:
    
    N *= 2
    I0, E0 = integ_atrap(f,N,I0)
    Ivals.append(I0)
    Evals.append(E0)
    Nvals.append(N)

hz = ones(len(Nvals))*1e-6

In [ ]:
from __future__ import division # python 2 compat
# "structured" adaptive version, translated from Racket
def _quad_simpsons_mem(f, a, fa, b, fb):
    """Evaluates the Simpson's Rule, also returning m and f(m) to reuse"""
    m = (a + b) / 2
    fm = f(m)
    return (m, fm, abs(b - a) / 6 * (fa + 4 * fm + fb))

def _quad_asr(f, a, fa, b, fb, eps, whole, m, fm):
    """
    Efficient recursive implementation of adaptive Simpson's rule.
    Function values at the start, middle, end of the intervals are retained.
    """
    lm, flm, left  = _quad_simpsons_mem(f, a, fa, m, fm)
    rm, frm, right = _quad_simpsons_mem(f, m, fm, b, fb)
    delta = left + right - whole
    if abs(delta) <= 15 * eps:
        return left + right + delta / 15
    return _quad_asr(f, a, fa, m, fm, eps/2, left , lm, flm) +\
           _quad_asr(f, m, fm, b, fb, eps/2, right, rm, frm)

def quad_asr(f, a, b, eps):
    """Integrate f from a to b using Adaptive Simpson's Rule with max error of eps."""
    fa, fb = f(a), f(b)
    m, fm, whole = _quad_simpsons_mem(f, a, fa, b, fb)
    return _quad_asr(f, a, fa, b, fb, eps, whole, m, fm)

from math import sin
print(quad_asr(sin, 0, 1, 1e-10))